# TRAIN HANGUL-RNN

In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import argparse
import time
import os
from six.moves import cPickle
from TextLoader import *
from Hangulpy import *
print ("Packages Imported")

Packages Imported


# LOAD DATASET WITH TEXTLOADER

In [2]:
corpus_name = "invisible_dragon" # "nine_dreams"

data_dir    = "data/" + corpus_name
batch_size  = 10
seq_length  = 100
data_loader = TextLoader(data_dir, batch_size, seq_length)

loading preprocessed files


# VOCAB AND CHARS

In [3]:
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader.vocab' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )

type of 'data_loader.vocab' is <type 'dict'>, length is 84
type of 'data_loader.chars' is <type 'tuple'>, length is 84


# VOCAB: DICTIONARY (CHAR->INDEX)

In [4]:
print (data_loader.vocab)

{u'_': 81, u'6': 63, u'|': 83, u'\n': 6, u'\r': 7, u',': 74, u'x': 75, u';': 73, u'[': 71, u'\u3144': 60, u'!': 28, u' ': 2, u'#': 68, u'"': 34, u'\u1d25': 0, u"'": 65, u')': 50, u'(': 51, u'+': 76, u'*': 82, u']': 72, u'\u3133': 58, u'/': 45, u'.': 24, u'\u3131': 5, u'0': 27, u'3': 54, u'2': 36, u'5': 61, u'\u3134': 4, u'\u3137': 11, u'\u3136': 49, u'\u3139': 8, u'\u3138': 31, u'\u3156': 52, u':': 40, u'\u313c': 67, u'?': 44, u'4': 59, u'\u3141': 14, u'\u3140': 77, u'\u3143': 57, u'\u3142': 21, u'\u3145': 15, u'7': 47, u'\u3147': 1, u'\u3146': 22, u'\u3149': 38, u'\u3148': 16, u'\u314b': 26, u'\u314a': 30, u'\u314d': 33, u'\u314c': 25, u'\u314f': 3, u'\u314e': 19, u'\u3151': 32, u'\u3150': 18, u'\u3153': 13, u'\u3152': 69, u'\u3155': 20, u'\u3154': 23, u'\u3157': 12, u'8': 46, u'\u3159': 62, u'\u3158': 41, u'\u315b': 29, u'\u315a': 53, u'\u315d': 48, u'\u315c': 17, u'\u315f': 39, u'^': 64, u'\u3161': 10, u'\u3160': 55, u'\u3163': 9, u'\u3162': 43, u'k': 78, u'9': 35, u'\u313a': 70, u'

# CHARS: LIST (INDEX->CHAR)

In [5]:
print (data_loader.chars)
# USAGE
print (data_loader.chars[0])

(u'\u1d25', u'\u3147', u' ', u'\u314f', u'\u3134', u'\u3131', u'\n', u'\r', u'\u3139', u'\u3163', u'\u3161', u'\u3137', u'\u3157', u'\u3153', u'\u3141', u'\u3145', u'\u3148', u'\u315c', u'\u3150', u'\u314e', u'\u3155', u'\u3142', u'\u3146', u'\u3154', u'.', u'\u314c', u'\u314b', u'0', u'!', u'\u315b', u'\u314a', u'\u3138', u'\u3151', u'\u314d', u'"', u'9', u'2', u'~', u'\u3149', u'\u315f', u':', u'\u3158', u'\u3132', u'\u3162', u'?', u'/', u'8', u'7', u'\u315d', u'\u3136', u')', u'(', u'\u3156', u'\u315a', u'3', u'\u3160', u'1', u'\u3143', u'\u3133', u'4', u'\u3144', u'5', u'\u3159', u'6', u'^', u"'", u'%', u'\u313c', u'#', u'\u3152', u'\u313a', u'[', u']', u';', u',', u'x', u'+', u'\u3140', u'k', u'<', u'}', u'_', u'*', u'|')
ᴥ


# TRAINING BATCH (IMPORTANT!!)

In [6]:
x, y = data_loader.next_batch()
print ("Type of 'x' is %s. Shape is %s" % (type(x), x.shape,))
print ("x looks like \n%s" % (x))
print
print ("Type of 'y' is %s. Shape is %s" % (type(y), y.shape,))
print ("y looks like \n%s" % (y))

Type of 'x' is <type 'numpy.ndarray'>. Shape is (50, 50)
x looks like 
[[ 7  6  1 ...,  3  0 37]
 [15 18  0 ...,  0 19  3]
 [38  3  0 ..., 61 50  7]
 ..., 
 [ 0  5  3 ...,  0  2 21]
 [ 8  0  2 ..., 12  5  0]
 [ 3  1  0 ..., 21 12  0]]

Type of 'y' is <type 'numpy.ndarray'>. Shape is (50, 50)
y looks like 
[[ 6  1 20 ...,  0 37  7]
 [18  0  5 ..., 19  3  4]
 [ 3  0  2 ..., 50  7  6]
 ..., 
 [ 5  3  0 ...,  2 21 12]
 [ 0  2 19 ...,  5  0 15]
 [ 1  0  1 ..., 12  0 11]]


# DEFINE A MULTILAYER LSTM NETWORK

In [7]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5. # <= GRADIENT CLIPPING (PRACTICALLY IMPORTANT)
vocab_size = data_loader.vocab_size

# SELECT RNN CELL (MULTI LAYER LSTM)
def unit_cell():
    return tf.contrib.rnn.BasicLSTMCell(rnn_size,state_is_tuple=True,reuse=tf.get_variable_scope().reuse)
cell = tf.contrib.rnn.MultiRNNCell([unit_cell() for _ in range(num_layers)])

# Set paths to the graph
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
targets    = tf.placeholder(tf.int32, [batch_size, seq_length])
initial_state = cell.zero_state(batch_size, tf.float32)

# Set Network
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
        inputs = tf.split(tf.nn.embedding_lookup(embedding, input_data), seq_length, 1)
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
print ("Network ready")

Network ready


# Define functions

In [8]:
# Output of RNN
outputs, last_state = tf.contrib.rnn.static_rnn(cell,inputs, initial_state, 
                                                scope='rnnlm')

output = tf.reshape(tf.concat(outputs,1), [-1, rnn_size])
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)

# Next word probability
probs = tf.nn.softmax(logits)
print ("FUNCTIONS READY")

FUNCTIONS READY


# DEFINE LOSS FUNCTION 

In [9]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], # Input
    [tf.reshape(targets, [-1])], # Target
    [tf.ones([batch_size * seq_length])], # Weight
    vocab_size)
print ("LOSS FUNCTION")

LOSS FUNCTION


# DEFINE COST FUNCTION 

In [10]:
cost = tf.reduce_sum(loss) / batch_size / seq_length

# GRADIENT CLIPPING ! 
lr = tf.Variable(0.0, trainable=False) # <= LEARNING RATE 
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
_optm = tf.train.AdamOptimizer(lr)
optm = _optm.apply_gradients(zip(grads, tvars))

final_state = last_state
print ("NETWORK READY")

NETWORK READY


# OPTIMIZE NETWORK WITH LR SCHEDULING

In [13]:
num_epochs    = 5000
save_every    = 100
learning_rate = 0.001
decay_rate    = 0.999

save_dir = 'data/' + corpus_name
sess = tf.InteractiveSession()

sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [14]:

summary_writer = tf.summary.FileWriter(save_dir
                    , graph=sess.graph)
saver = tf.train.Saver(tf.all_variables())
for e in range(num_epochs): # for all epochs
    # LEARNING RATE SCHEDULING 
    sess.run(tf.assign(lr, learning_rate * (decay_rate ** e)))

    data_loader.reset_batch_pointer()
    state = sess.run(initial_state)
    for b in range(data_loader.num_batches):
        start = time.time()
        x, y = data_loader.next_batch()
        feed = {input_data: x, targets: y, initial_state: state}
        # Train!
        train_loss, state, _ = sess.run([cost, final_state, optm], feed)
        end = time.time()
        # PRINT 
        if b % 100 == 0:
            print ("%d/%d (epoch: %d), loss: %.3f, time/batch: %.3f"  
                   % (e * data_loader.num_batches + b
                    , num_epochs * data_loader.num_batches
                    , e, train_loss, end - start))
        # SAVE MODEL
        if (e * data_loader.num_batches + b) % save_every == 0:
            checkpoint_path = os.path.join(save_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path
                       , global_step = e * data_loader.num_batches + b)
            print("model saved to {}".format(checkpoint_path))

Instructions for updating:
Please use tf.global_variables instead.
0/13000 (epoch: 0), loss: 4.413, time/batch: 2.158
model saved to data/invisible_dragon/model.ckpt
26/13000 (epoch: 1), loss: 3.472, time/batch: 1.443
model saved to data/invisible_dragon/model.ckpt
52/13000 (epoch: 2), loss: 3.389, time/batch: 1.694
78/13000 (epoch: 3), loss: 3.335, time/batch: 1.416
model saved to data/invisible_dragon/model.ckpt
104/13000 (epoch: 4), loss: 3.263, time/batch: 1.514
130/13000 (epoch: 5), loss: 3.160, time/batch: 1.650
model saved to data/invisible_dragon/model.ckpt
156/13000 (epoch: 6), loss: 3.063, time/batch: 1.368
182/13000 (epoch: 7), loss: 2.890, time/batch: 1.442
model saved to data/invisible_dragon/model.ckpt
208/13000 (epoch: 8), loss: 2.639, time/batch: 1.455
234/13000 (epoch: 9), loss: 2.356, time/batch: 1.575
model saved to data/invisible_dragon/model.ckpt
260/13000 (epoch: 10), loss: 2.219, time/batch: 1.415
286/13000 (epoch: 11), loss: 2.119, time/batch: 1.331
model saved 